<a href="https://colab.research.google.com/github/Pavel-Zinkevich/Quizz-bot/blob/main/db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/"

Mounted at /content/drive


In [ ]:
!pip install aiosqlite

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/config.ipynb"

In [ ]:
import aiosqlite


async def get_quiz_index(user_id):
    async with aiosqlite.connect(DB_NAME) as db:
        async with db.execute('SELECT question_index FROM quiz_state WHERE user_id = (?)', (user_id,)) as cursor:
            result = await cursor.fetchone()
            return result[0] if result else 0

async def get_quiz_score(user_id):
    async with aiosqlite.connect(DB_NAME) as db:
        async with db.execute('SELECT score FROM quiz_state WHERE user_id = ?', (user_id,)) as cursor:
            result = await cursor.fetchone()
            if result:
                return result[0]
            else:
                # Если записи нет, создаем запись с начальным значением 0
                await db.execute('INSERT INTO quiz_state (user_id, score, question_index) VALUES (?, ?, ?)', (user_id, 0, 0))
                await db.commit()
                return 0


async def update_score(user_id, score):
    async with aiosqlite.connect(DB_NAME) as db:
        print(f"Updating score for user {user_id} to {score}")  # Логируем обновление баллов
        await db.execute('''
        INSERT OR REPLACE INTO quiz_state (user_id, score) VALUES (?, ?)
        ''', (user_id, score))
        await db.commit()



async def update_quiz_index(user_id, index):
    async with aiosqlite.connect(DB_NAME) as db:
        # Используем UPDATE для обновления записи с заданным user_id
        await db.execute('''
            UPDATE quiz_state
            SET question_index = ?
            WHERE user_id = ?
        ''', (index, user_id))
        await db.commit()




async def create_table():
    async with aiosqlite.connect(DB_NAME) as db:
        await db.execute('''
        CREATE TABLE IF NOT EXISTS quiz_state (
            user_id INTEGER PRIMARY KEY,
            question_index INTEGER,
            score INTEGER DEFAULT 0
        )
        ''')
        await db.commit()
